## Imports

In [2]:
import pandas as pd
from string import ascii_uppercase as alfabeto
import pickle
from bs4 import BeautifulSoup
import requests

## Data extraction

### Node Definition

In [11]:
csv_file=('formula1_2020season_raceResults.csv')
def extract_data(csv):
    loaded_data=pd.read_csv(csv)
    return loaded_data
data=extract_data(csv_file)

In [ ]:
circuits = [
    "Austria",
    "Styria",
    "Hungary",
    "Great Britain",
    "70th Anniversary",
    "Spain",
    "Belgium",
    "Italy",
    "Tuscany",
    "Russia",
    "Eifel",
    "Portugal",
    "Emilia Romagna",
    "Turkey"
]
def transform_data(input_data, circuits):
    '''
        This function brings us the data and divide the data by race
    '''
    
    return transformed_data
transform_data(data)